# Run RIBBIT to detect _L. capito_  in Ichaway data 

This notebook is used to run RIBBIT to detect _Lithobates capito_ on Ichaway data. 

## How to use: 

1. Make sure all audio files end with the date/time for the file in the format "xxxxx_YYYYMMDD_HHMMSS.wav"
1. Edit and run all **Setup** code chunks to prepare the model parameters and file paths for use. 
  - Note:`#*#` indicates locations where you may want to edit (e.g. file paths, parameter values, etc.)

1. Use the **Run model** code chunks to run the model. 

### Data organization 

This assumes the data is organized such that there is a folder for each year and each year contains folders for each logger and each logger has subfolders for different dates. 

e.g. This would be the file path for the data from logger 1a in 2022 for the date 2-2-15: 

`/Volumes/Expansion/Frog Call Project/Calling Data/ichaway/ichaway_2022/1a/2-2-15`

### Tips and troubleshooting 
   
1. This code will take hours to run if you're doing a lot of data! Plug in your computer. Also make sure you computer will not turn off while it is running (you may need to do this in your computer settings). It's ok if the screen turns off, but the computer must continue running. The model saves results at checkpoints, so if it stops for some reason, some of the data will be saved. 

1. To try to make it faster: One option is to open multiple instances of this notebook and run different years and/or loggers seperately. If you do this, make sure the folders labels are different for each piece that is running. 

1. You may get (MANY) warnings that say something like `UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94`. This is fine to ignore - it's just saying the last spectrogram used by the model is shorter because the audio file ended. 

1. An example for running a univariate parameter exploration is given below, as well. 


### For more on using RIBBIT
[OpenSoundscape RIBBIT tutorial](http://opensoundscape.org/en/latest/tutorials/RIBBIT_pulse_rate_demo.html)


## Setup 

In [3]:
# run the file setup_functions.ipynb to define setting, import packages, and define functions 
%run ../ribbit_functions/setup_functions.ipynb

### Set base parameters 

These parameters can be adjusted to try to improve the model or to change the model for a different herp species 

#### Current best parameters for gopher frog detection (as of December 2022): 
base_pulse_rate_range = [14,28] pulses per sec

base_signal_band = [600,800] Hz

base_noise_bands = [[150,200], [1100,1200]] Hz

base_clip_duration = 1 seconds

base_clip_overlap = 0.5 seconds

base_window_samples = 512


In [4]:
# minimum and maximum rate of pulsing (pulses per second) to search for
base_pulse_rate_range = [14,28] #*#

# look for a vocalization in the range of 600-800 Hz
base_signal_band = [600,800] #*#


# subtract the amplitude signal from these frequency ranges
base_noise_bands = [[150,200], [1100,1200]] #*#

#divides the signal into segments this many seconds long, analyzes each independently
base_clip_duration = 1 #*# seconds
base_clip_overlap = 0.5 #*# seconds

# window_samples used to create spectrogram - default 512. smaller -> greater time resolution. bigger -> greater frequency resolution
base_window_samples = 512 #*#

#if True, it will show the power spectrum plot for each audio segment
show_plots = False # don't change this - will slow down model considerably 


### Define functions used to run RIBBIT

In [29]:
# run the file run_ribbit_functions.ipynb to define the functions used to run the RIBBIT model with different parameters 
# Note: this must be done after base parameters are set
%run ../ribbit_functions/run_ribbit_functions.ipynb


### Set up file paths

In [20]:
# folder path for where the audio files are stored 
folder_path = './sample_data/' #*# change this to your folder path containing your data. Should end in backslash
# e.g. Mayay's path: /Volumes/Expansion/Frog Call Project/Calling Data/ichaway/'

# folder label for where ribbit scores csv will be saved. folder will be named: ribbit_scores_folder_lab
folder_lab = '_sample_files' #*# change this to label the folder where the ribbit scores will be saved 

# edit this if you only want to run the model for specific years/loggers
all_years = ["2014", "2015"] #*# change this to choose years to analyze 
all_loggers = ['1a','2a','3a', '4a', '5a', '6a', '7a', '8a', '9a', '10a'] #*# change this to choose logger names to analyze



## Run model

In [32]:
%%capture --no-stdout --no-display 
# above used to stop warnings displaying

# create a df where the indices are all all file names we want to run 
ich_files = pd.DataFrame()
        
for i in range(0, len(all_years)):
    for j in range(0, len(all_loggers)):
        
        # folder name where data is stored
        folder = folder_path + 'ichaway_' + all_years[i] + '/' + all_loggers[j]  + '/*/*.wav'
        
        temp = pd.DataFrame(index = glob(folder),columns=['score', 'year', 'logger']) # create dataframe for year i, logger j
        temp['year'] = all_years[i] # automatically label year for these files
        temp['logger'] = all_loggers[j] # automatically label logger for these files
        
        ich_files = pd.concat([ich_files, temp])
        
# run ribbit for all of the files! 
ich_rs = ich_files.copy()
run_ribbit_for_indices(ich_rs, folder_lab = folder_lab)



Time elapsed  0:00:20.127718


## Example of running a univariate parameter space exploration

In [23]:
%%capture --no-stdout --no-display 
# above used to stop warnings displaying

ich_rs = ich_files.copy()[0:1]

# Testing various values of "pulse rate"
pulse_rate_vals = [[10,28], [12,28], [14,28]] #*# change these to pulse rate parameters we want to test 
run_ribbit_SA(ich_rs, test_param = "pulse_rate_range", param_vals = pulse_rate_vals) 


Time elapsed  0:00:01.120984
Time elapsed  0:00:01.202709
Time elapsed  0:00:01.231660
